In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd
import sqlalchemy as sa

In [2]:
# reload library without restarting kernel
%load_ext autoreload
%autoreload 2

In [3]:
my_run = BuildStockQuery(
    db_name="resstock_core", table_name="sdr_magic17", workgroup="rescore", buildstock_type="resstock"
)

INFO:buildstock_query.query_core:Loading sdr_magic17 ...


INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:42 queries cache read from None.
INFO:buildstock_query.query_core:42 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...
INFO:buildstock_query.query_core:No new queries to save.


         success  inapplicable  fail   Sum  Applied %  bad-chng  ok-chng  \
upgrade                                                                    
0           17.0           0.0   0.0  17.0        0.0         0        0   
1           17.0           0.0   0.0  17.0      100.0         0       17   
10          15.0           2.0   0.0  17.0       88.2         0       15   
11          17.0           0.0   0.0  17.0      100.0         0       17   
12          17.0           0.0   0.0  17.0      100.0         0       17   
13          17.0           0.0   0.0  17.0      100.0         0       17   
14          15.0           0.0   2.0  17.0      100.0         0       15   
15          15.0           2.0   0.0  17.0       88.2         0       15   
16          14.0           3.0   0.0  17.0       82.4         0       14   
17          17.0           0.0   0.0  17.0      100.0         0       17   
18          17.0           0.0   0.0  17.0      100.0        17        0   
19          

In [4]:
df1 = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'])
df2 = my_run.agg.query(
    enduses=["fuel_use_electricity_total_m_btu"], group_by=["time", "geometry_building_type_recs"])
pd.testing.assert_frame_equal(df1, df2)
df2

geometry_building_type_recs  sample_count   units_count  \
0                    Mobile Home             4  3.285812e+07   
1  Multi-Family with 2 - 4 Units             1  8.214531e+06   
2         Single-Family Detached            12  9.857437e+07   

   fuel_use_electricity_total_m_btu  
0                      5.676734e+08  
1                      1.108633e+08  
2                      4.080141e+09

In [5]:
df1 = my_run.agg.aggregate_annual(
    enduses=["fuel_use_electricity_total_m_btu"],
    group_by=["geometry_building_type_recs"],
    agg_func="max",
    get_nonzero_count=True,
)
df2 = my_run.agg.query(
    enduses=["fuel_use_electricity_total_m_btu"],
    group_by=["geometry_building_type_recs"],
    agg_func="max",
    get_nonzero_count=True,
)
pd.testing.assert_frame_equal(df1, df2)
df2
    

geometry_building_type_recs  sample_count   units_count  \
0                    Mobile Home             4  3.285812e+07   
1  Multi-Family with 2 - 4 Units             1  8.214531e+06   
2         Single-Family Detached            12  9.857437e+07   

   fuel_use_electricity_total_m_btu__max  \
0                                 28.436   
1                                 13.496   
2                                 83.275   

   fuel_use_electricity_total_m_btu__nonzero_units_count  
0                                       3.285812e+07      
1                                       8.214531e+06      
2                                       9.857437e+07

In [6]:
from sqlalchemy.sql import func as safunc
import sqlalchemy as sa

df1 = my_run.agg.aggregate_annual(
    enduses=["fuel_use_natural_gas_total_m_btu"],
    group_by=["geometry_building_type_recs"],
    get_nonzero_count=True,
)
df2 = my_run.agg.query(
    enduses=["fuel_use_natural_gas_total_m_btu"],
    group_by=["geometry_building_type_recs"],
    get_nonzero_count=True,
)
pd.testing.assert_frame_equal(df1, df2)
print(df2)

     geometry_building_type_recs  sample_count   units_count  \
0                    Mobile Home             4  3.285812e+07   
1  Multi-Family with 2 - 4 Units             1  8.214531e+06   
2         Single-Family Detached            12  9.857437e+07   

   fuel_use_natural_gas_total_m_btu  \
0                      2.084026e+08   
1                      0.000000e+00   
2                      6.231329e+09   

   fuel_use_natural_gas_total_m_btu__nonzero_units_count  
0                                       3.285812e+07      
1                                       0.000000e+00      
2                                       9.035984e+07      


In [7]:
baseline_agg = my_run.agg.aggregate_annual(
    enduses=["fuel_use_electricity_total_m_btu"], group_by=["geometry_building_type_recs"], get_nonzero_count=True
)
baseline_agg

geometry_building_type_recs  sample_count   units_count  \
0                    Mobile Home             4  3.285812e+07   
1  Multi-Family with 2 - 4 Units             1  8.214531e+06   
2         Single-Family Detached            12  9.857437e+07   

   fuel_use_electricity_total_m_btu  \
0                      5.676734e+08   
1                      1.108633e+08   
2                      4.080141e+09   

   fuel_use_electricity_total_m_btu__nonzero_units_count  
0                                       3.285812e+07      
1                                       8.214531e+06      
2                                       9.857437e+07

In [8]:
my_run.save_cache()

INFO:buildstock_query.query_core:No new queries to save.


In [9]:
all_enduses = my_run.get_cols(my_run.ts_table)
all_enduses = [str(x) for x in all_enduses if "end_use__" in str(x)]
all_enduses

['sdr_magic17_timeseries.end_use__electricity__ceiling_fan__kwh',
 'sdr_magic17_timeseries.end_use__electricity__clothes_dryer__kwh',
 'sdr_magic17_timeseries.end_use__electricity__clothes_washer__kwh',
 'sdr_magic17_timeseries.end_use__electricity__cooling__kwh',
 'sdr_magic17_timeseries.end_use__electricity__cooling_fans_pumps__kwh',
 'sdr_magic17_timeseries.end_use__electricity__dishwasher__kwh',
 'sdr_magic17_timeseries.end_use__electricity__freezer__kwh',
 'sdr_magic17_timeseries.end_use__electricity__heating__kwh',
 'sdr_magic17_timeseries.end_use__electricity__heating_fans_pumps__kwh',
 'sdr_magic17_timeseries.end_use__electricity__heating_heat_pump_backup__kwh',
 'sdr_magic17_timeseries.end_use__electricity__heating_heat_pump_backup_fans_pumps__kwh',
 'sdr_magic17_timeseries.end_use__electricity__hot_water__kwh',
 'sdr_magic17_timeseries.end_use__electricity__lighting_exterior__kwh',
 'sdr_magic17_timeseries.end_use__electricity__lighting_garage__kwh',
 'sdr_magic17_timeseries.

In [10]:
df1 = my_run.agg.aggregate_timeseries(
    enduses=["fuel_use__electricity__total__kwh"],
    restrict=[("build_existing_model.state", ["TX"])],
    timestamp_grouping_func="month",
    group_by=["geometry_building_type_recs", "build_existing_model.state"],
    get_query_only=False,
)
df2 = my_run.agg.query(
    annual_only=False,
    timestamp_grouping_func="month",
    enduses=["fuel_use__electricity__total__kwh"],
    restrict=[("build_existing_model.state", ["TX"])],
    group_by=["geometry_building_type_recs", "build_existing_model.state", "time"],
    get_query_only=False,
)
pd.testing.assert_frame_equal(df1, df2)
df2

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


geometry_building_type_recs state       time  sample_count   units_count  \
0       Single-Family Detached    TX 2018-01-01             1  8.214531e+06   
1       Single-Family Detached    TX 2018-02-01             1  8.214531e+06   
2       Single-Family Detached    TX 2018-03-01             1  8.214531e+06   
3       Single-Family Detached    TX 2018-04-01             1  8.214531e+06   
4       Single-Family Detached    TX 2018-05-01             1  8.214531e+06   
5       Single-Family Detached    TX 2018-06-01             1  8.214531e+06   
6       Single-Family Detached    TX 2018-07-01             1  8.214531e+06   
7       Single-Family Detached    TX 2018-08-01             1  8.214531e+06   
8       Single-Family Detached    TX 2018-09-01             1  8.214531e+06   
9       Single-Family Detached    TX 2018-10-01             1  8.214531e+06   
10      Single-Family Detached    TX 2018-11-01             1  8.214531e+06   
11      Single-Family Detached    TX 2018-12-01             1  8.214531e+06   

    rows_per_sample  fuel_use__electricity__total__kwh  
0              2976                       4.977332e+09  
1              2688                       4.449910e+09  
2              2976                       5.432154e+09  
3              2880                       6.623286e+09  
4              2976                       9.696399e+09  
5              2880                       1.212763e+10  
6              2976                       1.435680e+10  
7              2976                       1.462632e+10  
8              2880                       1.267576e+10  
9              2976                       8.344871e+09  
10             2880                       6.286942e+09  
11             2976                       4.957790e+09

In [11]:
# peak electricity use for each building in AL
df1 = my_run.agg.aggregate_timeseries(
    enduses=["fuel_use__electricity__total__kwh"],
    collapse_ts=True,
    agg_func="max",
    group_by=[my_run.bs_bldgid_column],
    get_query_only=False,
)
df2 = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="year",
    agg_func="max",
    group_by=[my_run.bs_bldgid_column],
    get_query_only=False,
)
pd.testing.assert_frame_equal(df1, df2)
df2

INFO:buildstock_query.aggregate_query:Aggregation done across timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


building_id  sample_count   units_count  \
0          1958             1  8.214531e+06   
1         14180             1  8.214531e+06   
2         70326             1  8.214531e+06   
3        113940             1  8.214531e+06   
4        123855             1  8.214531e+06   
5        144662             1  8.214531e+06   
6        162498             1  8.214531e+06   
7        208457             1  8.214531e+06   
8        229246             1  8.214531e+06   
9        293863             1  8.214531e+06   
10       294353             1  8.214531e+06   
11       346549             1  8.214531e+06   
12       355223             1  8.214531e+06   
13       357547             1  8.214531e+06   
14       367274             1  8.214531e+06   
15       442952             1  8.214531e+06   
16       529000             1  8.214531e+06   

    fuel_use__electricity__total__kwh__max  
0                                    0.519  
1                                    3.071  
2                                    2.134  
3                                    0.567  
4                                    1.557  
5                                    2.454  
6                                    3.347  
7                                    0.839  
8                                    2.863  
9                                    2.215  
10                                   3.402  
11                                   1.417  
12                                   2.419  
13                                   0.922  
14                                   1.953  
15                                   1.993  
16                                   0.581

In [12]:
df1 = my_run.savings.savings_shape(
    upgrade_id=2,
    enduses=["fuel_use__electricity__total__kwh"],
    restrict=[("build_existing_model.state", ["TX"])],
    annual_only=False,
)
df2 = my_run.agg.query(
    upgrade_id=2,
    include_savings=True,
    include_baseline=True,
    include_upgrade=False,
    enduses=["fuel_use__electricity__total__kwh"],
    restrict=[("build_existing_model.state", ["TX"])],
    annual_only=False,
)
pd.testing.assert_frame_equal(df1, df2)
df2

time  sample_count   units_count  \
0     2018-01-01 00:15:00             1  8.214531e+06   
1     2018-01-01 00:30:00             1  8.214531e+06   
2     2018-01-01 00:45:00             1  8.214531e+06   
3     2018-01-01 01:00:00             1  8.214531e+06   
4     2018-01-01 01:15:00             1  8.214531e+06   
...                   ...           ...           ...   
35035 2018-12-31 23:00:00             1  8.214531e+06   
35036 2018-12-31 23:15:00             1  8.214531e+06   
35037 2018-12-31 23:30:00             1  8.214531e+06   
35038 2018-12-31 23:45:00             1  8.214531e+06   
35039 2019-01-01 00:00:00             1  8.214531e+06   

       fuel_use__electricity__total__kwh__baseline  \
0                                     1.141820e+06   
1                                     1.108962e+06   
2                                     9.775291e+05   
3                                     9.035984e+05   
4                                     9.200274e+05   
...                                            ...   
35035                                 1.511474e+06   
35036                                 1.495045e+06   
35037                                 1.421114e+06   
35038                                 1.117176e+06   
35039                                 1.117176e+06   

       fuel_use__electricity__total__kwh__savings  
0                                   -2.267210e+06  
1                                   -2.513646e+06  
2                                   -2.669722e+06  
3                                   -2.809369e+06  
4                                   -2.875086e+06  
...                                           ...  
35035                               -8.214531e+03  
35036                                0.000000e+00  
35037                                0.000000e+00  
35038                                0.000000e+00  
35039                                0.000000e+00  

[35040 rows x 5 columns]

In [13]:
df2 = my_run.agg.query(
    upgrade_id=2,
    include_savings=False,
    include_baseline=False,
    include_upgrade=True,
    enduses=["fuel_use__electricity__total__kwh"],
    restrict=[("build_existing_model.state", ["TX"])],
    annual_only=False,
)
df2

time  sample_count   units_count  \
0     2018-01-01 00:15:00             1  8.214531e+06   
1     2018-01-01 00:30:00             1  8.214531e+06   
2     2018-01-01 00:45:00             1  8.214531e+06   
3     2018-01-01 01:00:00             1  8.214531e+06   
4     2018-01-01 01:15:00             1  8.214531e+06   
...                   ...           ...           ...   
35035 2018-12-31 23:00:00             1  8.214531e+06   
35036 2018-12-31 23:15:00             1  8.214531e+06   
35037 2018-12-31 23:30:00             1  8.214531e+06   
35038 2018-12-31 23:45:00             1  8.214531e+06   
35039 2019-01-01 00:00:00             1  8.214531e+06   

       fuel_use__electricity__total__kwh  
0                           3.409030e+06  
1                           3.622608e+06  
2                           3.647252e+06  
3                           3.712968e+06  
4                           3.795113e+06  
...                                  ...  
35035                       1.519688e+06  
35036                       1.495045e+06  
35037                       1.421114e+06  
35038                       1.117176e+06  
35039                       1.117176e+06  

[35040 rows x 4 columns]

In [14]:
df1 = my_run.savings.savings_shape(
    upgrade_id=2,
    enduses=["fuel_use__electricity__total__kwh"],
    group_by=["geometry_building_type_recs"],
    annual_only=False,
)
df2 = my_run.agg.query(
    annual_only=False,
    upgrade_id=2,
    include_savings=True,
    include_baseline=True,
    include_upgrade=False,
    enduses=["fuel_use__electricity__total__kwh"],
    group_by=["geometry_building_type_recs"],
)
pd.testing.assert_frame_equal(df1, df2)
df2


time  sample_count   units_count  \
0      2018-01-01 00:15:00             4  3.285812e+07   
1      2018-01-01 00:30:00             4  3.285812e+07   
2      2018-01-01 00:45:00             4  3.285812e+07   
3      2018-01-01 01:00:00             4  3.285812e+07   
4      2018-01-01 01:15:00             4  3.285812e+07   
...                    ...           ...           ...   
105115 2018-12-31 23:00:00            12  9.857437e+07   
105116 2018-12-31 23:15:00            12  9.857437e+07   
105117 2018-12-31 23:30:00            12  9.857437e+07   
105118 2018-12-31 23:45:00            12  9.857437e+07   
105119 2019-01-01 00:00:00            12  9.857437e+07   

        fuel_use__electricity__total__kwh__baseline  \
0                                      2.743653e+06   
1                                      2.751868e+06   
2                                      2.751868e+06   
3                                      2.751868e+06   
4                                      2.768297e+06   
...                                             ...   
105115                                 2.566219e+07   
105116                                 1.845805e+07   
105117                                 2.895622e+07   
105118                                 1.808840e+07   
105119                                 1.655228e+07   

        fuel_use__electricity__total__kwh__savings geometry_building_type_recs  
0                                    -1.199321e+07                 Mobile Home  
1                                    -1.273252e+07                 Mobile Home  
2                                    -1.373470e+07                 Mobile Home  
3                                    -1.499152e+07                 Mobile Home  
4                                    -1.495866e+07                 Mobile Home  
...                                            ...                         ...  
105115                               -5.576023e+07      Single-Family Detached  
105116                               -5.626953e+07      Single-Family Detached  
105117                               -5.470877e+07      Single-Family Detached  
105118                               -5.954713e+07      Single-Family Detached  
105119                               -6.031108e+07      Single-Family Detached  

[105120 rows x 6 columns]

In [15]:
df1 = my_run.savings.savings_shape(
    upgrade_id=2,
    enduses=["fuel_use__electricity__total__kwh"],
    group_by=["geometry_building_type_recs"],
    get_quartiles=True,
    annual_only=False,
)
df2 = my_run.agg.query(
    upgrade_id=2,
    include_savings=True,
    include_baseline=True,
    include_upgrade=False,
    enduses=["fuel_use__electricity__total__kwh"],
    group_by=["geometry_building_type_recs"],
    get_quartiles=True,
    annual_only=False,
)
pd.testing.assert_frame_equal(df1, df2.drop(columns=["fuel_use__electricity__total__kwh__baseline__quartiles"]))
df2

time  sample_count   units_count  \
0      2018-01-01 00:15:00             4  3.285812e+07   
1      2018-01-01 00:30:00             4  3.285812e+07   
2      2018-01-01 00:45:00             4  3.285812e+07   
3      2018-01-01 01:00:00             4  3.285812e+07   
4      2018-01-01 01:15:00             4  3.285812e+07   
...                    ...           ...           ...   
105115 2018-12-31 23:00:00            12  9.857437e+07   
105116 2018-12-31 23:15:00            12  9.857437e+07   
105117 2018-12-31 23:30:00            12  9.857437e+07   
105118 2018-12-31 23:45:00            12  9.857437e+07   
105119 2019-01-01 00:00:00            12  9.857437e+07   

        fuel_use__electricity__total__kwh__baseline  \
0                                      2.743653e+06   
1                                      2.751868e+06   
2                                      2.751868e+06   
3                                      2.751868e+06   
4                                      2.768297e+06   
...                                             ...   
105115                                 2.566219e+07   
105116                                 1.845805e+07   
105117                                 2.895622e+07   
105118                                 1.808840e+07   
105119                                 1.655228e+07   

        fuel_use__electricity__total__kwh__savings  \
0                                    -1.199321e+07   
1                                    -1.273252e+07   
2                                    -1.373470e+07   
3                                    -1.499152e+07   
4                                    -1.495866e+07   
...                                            ...   
105115                               -5.576023e+07   
105116                               -5.626953e+07   
105117                               -5.470877e+07   
105118                               -5.954713e+07   
105119                               -6.031108e+07   

       fuel_use__electricity__total__kwh__baseline__quartiles  \
0            [0.05, 0.05, 0.058, 0.106, 0.12, 0.12, 0.12]       
1         [0.05, 0.05, 0.058, 0.106, 0.121, 0.121, 0.121]       
2         [0.05, 0.05, 0.059, 0.104, 0.122, 0.122, 0.122]       
3         [0.05, 0.05, 0.059, 0.103, 0.123, 0.123, 0.123]       
4           [0.051, 0.051, 0.06, 0.106, 0.12, 0.12, 0.12]       
...                                                   ...       
105115  [0.045, 0.045, 0.139, 0.202, 0.319, 0.689, 0.689]       
105116  [0.079, 0.079, 0.137, 0.161, 0.201, 0.465, 0.465]       
105117  [0.079, 0.079, 0.137, 0.173, 0.331, 1.298, 1.298]       
105118  [0.043, 0.043, 0.136, 0.138, 0.305, 0.422, 0.422]       
105119   [0.043, 0.043, 0.12, 0.138, 0.201, 0.409, 0.409]       

       fuel_use__electricity__total__kwh__savings__quartiles  \
0       [-0.708, -0.708, -0.42900000000000005, -0.173,...      
1       [-0.752, -0.752, -0.469, -0.174, -0.155, -0.15...      
2       [-0.797, -0.797, -0.541, -0.176, -0.158, -0.15...      
3       [-0.846, -0.846, -0.637, -0.178, -0.164, -0.16...      
4       [-0.849, -0.849, -0.636, -0.17300000000000001,...      
...                                                   ...      
105115  [-1.999, -1.999, -0.846, -0.342, -0.127, 0.274...      
105116  [-2.0300000000000002, -2.0300000000000002, -0....      
105117  [-2.061, -2.061, -0.8220000000000001, -0.34800...      
105118  [-2.094, -2.094, -0.8210000000000001, -0.35000...      
105119  [-2.127, -2.127, -0.8160000000000001, -0.35200...      

       geometry_building_type_recs  
0                      Mobile Home  
1                      Mobile Home  
2                      Mobile Home  
3                      Mobile Home  
4                      Mobile Home  
...                            ...  
105115      Single-Family Detached  
105116      Single-Family Detached  
105117      Single-Family Detached  
105118      Single-Family Detached  
105119      Single-Family Detached  

[105120 rows x 8 col

In [16]:
df = my_run.agg.query(enduses=["fuel_use_electricity_total_m_btu"], upgrade_id="10", group_by=["building_id"], applied_only=True)
upgrade_10_applied_bldgs = my_run.execute(f"select distinct(building_id) from {my_run.up_table.name} where upgrade = '10' and completed_status = 'Success'")
assert sorted(df['building_id']) == sorted(upgrade_10_applied_bldgs['building_id'])
df

building_id  sample_count   units_count  fuel_use_electricity_total_m_btu
0          1958             1  8.214531e+06                      1.118080e+08
1         14180             1  8.214531e+06                      5.235038e+08
2        123855             1  8.214531e+06                      2.734946e+08
3        144662             1  8.214531e+06                      4.278128e+08
4        162498             1  8.214531e+06                      6.173959e+08
5        208457             1  8.214531e+06                      1.292721e+08
6        229246             1  8.214531e+06                      8.764247e+08
7        293863             1  8.214531e+06                      3.501115e+08
8        294353             1  8.214531e+06                      6.040062e+08
9        346549             1  8.214531e+06                      3.241700e+08
10       355223             1  8.214531e+06                      3.304048e+08
11       357547             1  8.214531e+06                      3.107228e+08
12       367274             1  8.214531e+06                      4.467637e+08
13       442952             1  8.214531e+06                      4.239930e+08
14       529000             1  8.214531e+06                      6.110789e+07

In [17]:
df = my_run.agg.query(enduses=["fuel_use_electricity_total_m_btu"], upgrade_id="10", group_by=["building_id"], applied_only=False)
upgrade_10_all_bldgs = my_run.execute(f"select distinct(building_id) from {my_run.up_table.name} where upgrade = '10'")
assert sorted(df['building_id']) == sorted(upgrade_10_all_bldgs['building_id'])
df

building_id  sample_count   units_count  fuel_use_electricity_total_m_btu
0          1958             1  8.214531e+06                      1.118080e+08
1         14180             1  8.214531e+06                      5.235038e+08
2         70326             1  8.214531e+06                      1.108633e+08
3        113940             1  8.214531e+06                      1.197350e+08
4        123855             1  8.214531e+06                      2.734946e+08
5        144662             1  8.214531e+06                      4.278128e+08
6        162498             1  8.214531e+06                      6.173959e+08
7        208457             1  8.214531e+06                      1.292721e+08
8        229246             1  8.214531e+06                      8.764247e+08
9        293863             1  8.214531e+06                      3.501115e+08
10       294353             1  8.214531e+06                      6.040062e+08
11       346549             1  8.214531e+06                      3.241700e+08
12       355223             1  8.214531e+06                      3.304048e+08
13       357547             1  8.214531e+06                      3.107228e+08
14       367274             1  8.214531e+06                      4.467637e+08
15       442952             1  8.214531e+06                      4.239930e+08
16       529000             1  8.214531e+06                      6.110789e+07

In [18]:
my_run.save_cache()

INFO:buildstock_query.query_core:No new queries to save.


In [19]:
ts_df = my_run.agg.query(annual_only=False, enduses=["fuel_use__electricity__total__kwh"],
include_savings=False, include_baseline=False, include_upgrade=True,
timestamp_grouping_func="month", upgrade_id="10", group_by=["building_id"], applied_only=True,
get_query_only=False)
ts_df2 = my_run.agg.aggregate_timeseries(enduses=["fuel_use__electricity__total__kwh"],
timestamp_grouping_func="month", upgrade_id="10", group_by=["time", "building_id"],
get_query_only=False)
pd.testing.assert_frame_equal(ts_df, ts_df2)
assert sorted(set(ts_df['building_id'].to_list())) == sorted(upgrade_10_applied_bldgs['building_id'].to_list())
ts_df

INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 10.


time  building_id  sample_count   units_count  rows_per_sample  \
0   2018-01-01         1958             1  8.214531e+06             2976   
1   2018-01-01        14180             1  8.214531e+06             2976   
2   2018-01-01       123855             1  8.214531e+06             2976   
3   2018-01-01       144662             1  8.214531e+06             2976   
4   2018-01-01       162498             1  8.214531e+06             2976   
..         ...          ...           ...           ...              ...   
175 2018-12-01       355223             1  8.214531e+06             2976   
176 2018-12-01       357547             1  8.214531e+06             2976   
177 2018-12-01       367274             1  8.214531e+06             2976   
178 2018-12-01       442952             1  8.214531e+06             2976   
179 2018-12-01       529000             1  8.214531e+06             2976   

     fuel_use__electricity__total__kwh  
0                         5.496761e+09  
1                         1.830001e+10  
2                         1.196657e+10  
3                         1.620343e+10  
4                         8.254001e+09  
..                                 ...  
175                       9.717855e+09  
176                       9.932674e+09  
177                       1.876987e+10  
178                       1.818770e+10  
179                       2.867142e+09  

[180 rows x 6 columns]

In [20]:
ts_df_applied = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="10",
    group_by=["building_id", "time"],
    applied_only=True,
)
assert sorted(set(ts_df_applied['building_id'].to_list())) == sorted(upgrade_10_applied_bldgs['building_id'].to_list())
ts_df_applied

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id       time  sample_count   units_count  rows_per_sample  \
0           1958 2018-01-01             1  8.214531e+06             2976   
1           1958 2018-02-01             1  8.214531e+06             2688   
2           1958 2018-03-01             1  8.214531e+06             2976   
3           1958 2018-04-01             1  8.214531e+06             2880   
4           1958 2018-05-01             1  8.214531e+06             2976   
..           ...        ...           ...           ...              ...   
175       529000 2018-08-01             1  8.214531e+06             2976   
176       529000 2018-09-01             1  8.214531e+06             2880   
177       529000 2018-10-01             1  8.214531e+06             2976   
178       529000 2018-11-01             1  8.214531e+06             2880   
179       529000 2018-12-01             1  8.214531e+06             2976   

     fuel_use__electricity__total__kwh  
0                         5.496761e+09  
1                         4.897766e+09  
2                         3.693927e+09  
3                         1.732477e+09  
4                         1.419438e+09  
..                                 ...  
175                       1.968719e+09  
176                       1.164344e+09  
177                       7.619963e+08  
178                       8.231945e+08  
179                       2.867142e+09  

[180 rows x 6 columns]

In [21]:
ts_df_all = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="10",
    group_by=["building_id", "time"],
    applied_only=False,
)
assert sorted(set(ts_df_all['building_id'].to_list())) == sorted(upgrade_10_all_bldgs['building_id'].to_list())
ts_df_all

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id       time  sample_count   units_count  rows_per_sample  \
0           1958 2018-01-01             1  8.214531e+06             2976   
1           1958 2018-02-01             1  8.214531e+06             2688   
2           1958 2018-03-01             1  8.214531e+06             2976   
3           1958 2018-04-01             1  8.214531e+06             2880   
4           1958 2018-05-01             1  8.214531e+06             2976   
..           ...        ...           ...           ...              ...   
199       529000 2018-08-01             1  8.214531e+06             2976   
200       529000 2018-09-01             1  8.214531e+06             2880   
201       529000 2018-10-01             1  8.214531e+06             2976   
202       529000 2018-11-01             1  8.214531e+06             2880   
203       529000 2018-12-01             1  8.214531e+06             2976   

     fuel_use__electricity__total__kwh  
0                         5.496761e+09  
1                         4.897766e+09  
2                         3.693927e+09  
3                         1.732477e+09  
4                         1.419438e+09  
..                                 ...  
199                       1.968719e+09  
200                       1.164344e+09  
201                       7.619963e+08  
202                       8.231945e+08  
203                       2.867142e+09  

[204 rows x 6 columns]

In [22]:
unapplied_bldgs = list(set(upgrade_10_all_bldgs['building_id']) - set(upgrade_10_applied_bldgs['building_id']))
ubldg = int(unapplied_bldgs[0])
ubldg

113940

In [23]:
# Verify that the baseline building included in ts_df_all has the same value as direct baseline query for the building
ts_df_baseline = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="0",
    group_by=["building_id", "time"],
    restrict=[("building_id", ubldg)],
)
ts_df_baseline_2 = my_run.agg.aggregate_timeseries(
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="0",
    group_by=["building_id", "time"],
    restrict=[("building_id", ubldg)],
)
pd.testing.assert_frame_equal(ts_df_baseline, ts_df_baseline_2)
pd.testing.assert_frame_equal(ts_df_baseline, ts_df_all[ts_df_all["building_id"] == ubldg].reset_index(drop=True))
ts_df_baseline

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.
INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id       time  sample_count   units_count  rows_per_sample  \
0        113940 2018-01-01             1  8.214531e+06             2976   
1        113940 2018-02-01             1  8.214531e+06             2688   
2        113940 2018-03-01             1  8.214531e+06             2976   
3        113940 2018-04-01             1  8.214531e+06             2880   
4        113940 2018-05-01             1  8.214531e+06             2976   
5        113940 2018-06-01             1  8.214531e+06             2880   
6        113940 2018-07-01             1  8.214531e+06             2976   
7        113940 2018-08-01             1  8.214531e+06             2976   
8        113940 2018-09-01             1  8.214531e+06             2880   
9        113940 2018-10-01             1  8.214531e+06             2976   
10       113940 2018-11-01             1  8.214531e+06             2880   
11       113940 2018-12-01             1  8.214531e+06             2976   

    fuel_use__electricity__total__kwh  
0                        2.718985e+09  
1                        2.517088e+09  
2                        2.651018e+09  
3                        2.436118e+09  
4                        2.489331e+09  
5                        3.442899e+09  
6                        4.198972e+09  
7                        3.779744e+09  
8                        3.060397e+09  
9                        2.512578e+09  
10                       2.481799e+09  
11                       2.803480e+09

In [24]:
ts_df_applied_with_baseline = my_run.agg.query(
    annual_only=False,
    include_baseline=True,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="10",
    group_by=["building_id", "time"],
    applied_only=True,
)
ts_df_applied_with_baseline

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id       time  sample_count   units_count  rows_per_sample  \
0           1958 2018-01-01             1  8.214531e+06             2976   
1           1958 2018-02-01             1  8.214531e+06             2688   
2           1958 2018-03-01             1  8.214531e+06             2976   
3           1958 2018-04-01             1  8.214531e+06             2880   
4           1958 2018-05-01             1  8.214531e+06             2976   
..           ...        ...           ...           ...              ...   
175       529000 2018-08-01             1  8.214531e+06             2976   
176       529000 2018-09-01             1  8.214531e+06             2880   
177       529000 2018-10-01             1  8.214531e+06             2976   
178       529000 2018-11-01             1  8.214531e+06             2880   
179       529000 2018-12-01             1  8.214531e+06             2976   

     fuel_use__electricity__total__kwh__baseline  \
0                                   2.041648e+09   
1                                   1.832259e+09   
2                                   1.824291e+09   
3                                   1.469637e+09   
4                                   1.410164e+09   
..                                           ...   
175                                 2.476377e+09   
176                                 1.353106e+09   
177                                 8.679555e+08   
178                                 8.818874e+08   
179                                 1.267518e+09   

     fuel_use__electricity__total__kwh__upgrade  
0                                  5.496761e+09  
1                                  4.897766e+09  
2                                  3.693927e+09  
3                                  1.732477e+09  
4                                  1.419438e+09  
..                                          ...  
175                                1.968719e+09  
176                                1.164344e+09  
177                                7.619963e+08  
178                                8.231945e+08  
179                                2.867142e+09  

[180 rows x 7 columns]

In [25]:
# verify the 
ts_df_other = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="month",
    upgrade_id="0",
    group_by=["building_id", "time"],
    avoid=[("building_id", [int(u) for u in unapplied_bldgs])],
).rename(columns={"fuel_use__electricity__total__kwh": "fuel_use__electricity__total__kwh__baseline"})
pd.testing.assert_frame_equal(ts_df_other[['fuel_use__electricity__total__kwh__baseline']],
                              ts_df_applied_with_baseline[['fuel_use__electricity__total__kwh__baseline']],
                              )
ts_df_other


INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id       time  sample_count   units_count  rows_per_sample  \
0           1958 2018-01-01             1  8.214531e+06             2976   
1           1958 2018-02-01             1  8.214531e+06             2688   
2           1958 2018-03-01             1  8.214531e+06             2976   
3           1958 2018-04-01             1  8.214531e+06             2880   
4           1958 2018-05-01             1  8.214531e+06             2976   
..           ...        ...           ...           ...              ...   
175       529000 2018-08-01             1  8.214531e+06             2976   
176       529000 2018-09-01             1  8.214531e+06             2880   
177       529000 2018-10-01             1  8.214531e+06             2976   
178       529000 2018-11-01             1  8.214531e+06             2880   
179       529000 2018-12-01             1  8.214531e+06             2976   

     fuel_use__electricity__total__kwh__baseline  
0                                   2.041648e+09  
1                                   1.832259e+09  
2                                   1.824291e+09  
3                                   1.469637e+09  
4                                   1.410164e+09  
..                                           ...  
175                                 2.476377e+09  
176                                 1.353106e+09  
177                                 8.679555e+08  
178                                 8.818874e+08  
179                                 1.267518e+09  

[180 rows x 6 columns]

In [48]:
df = my_run.agg.query(
    annual_only=False,
    enduses=["fuel_use__electricity__total__kwh"],
    timestamp_grouping_func="year",
    upgrade_id="0",
    group_by=["building_id", "time"],
    avoid=[("building_id", [int(u) for u in unapplied_bldgs])],
    limit=10,
    get_nonzero_count=False,
    sort=True,
)
assert len(df) == 10
df


INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id  sample_count   units_count  fuel_use__electricity__total__kwh
0         1958             1  8.214531e+06                       2.127208e+10
1        14180             1  8.214531e+06                       1.472858e+11
2       123855             1  8.214531e+06                       5.289458e+10
3       144662             1  8.214531e+06                       1.047316e+11
4       162498             1  8.214531e+06                       2.004805e+11
5       208457             1  8.214531e+06                       2.374470e+10
6       229246             1  8.214531e+06                       1.475884e+11
7       293863             1  8.214531e+06                       1.168950e+11
8       294353             1  8.214531e+06                       1.129665e+11
9       346549             1  8.214531e+06                       1.045552e+11

In [52]:
df = my_run.agg.query(
    annual_only=True,
    enduses=["fuel_use_natural_gas_total_m_btu"],
    upgrade_id="1",
    group_by=[my_run.up_bldgid_column, "time"],
    limit=10,
    get_nonzero_count=False,
    sort=True,
)
assert len(df) == 10
df

INFO:botocore.httpchecksum:Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


building_id  sample_count   units_count  fuel_use_natural_gas_total_m_btu
0         1958             1  8.214531e+06                      3.105093e+07
1        14180             1  8.214531e+06                      2.857917e+08
2        70326             1  8.214531e+06                      0.000000e+00
3       113940             1  8.214531e+06                      2.172415e+08
4       123855             1  8.214531e+06                      5.352588e+07
5       144662             1  8.214531e+06                      7.612405e+07
6       162498             1  8.214531e+06                      1.013098e+08
7       208457             1  8.214531e+06                      3.607822e+07
8       229246             1  8.214531e+06                      1.130237e+08
9       293863             1  8.214531e+06                      4.125337e+07